In [40]:
import pandas as pd
df = pd.read_excel('seoul_list.xlsx')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11076 entries, 0 to 11075
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   11076 non-null  int64 
 1   운영기관         11076 non-null  object
 2   충전소          11076 non-null  object
 3   충전기 ID       11076 non-null  int64 
 4   타입           11076 non-null  object
 5   지역           11076 non-null  object
 6   시군구          11071 non-null  object
 7   주소           11076 non-null  object
 8   이용가능시간       10272 non-null  object
 9   제한           5850 non-null   object
 10  종류           296 non-null    object
 11  비고           1804 non-null   object
 12  nan          9572 non-null   object
 13  Unnamed: 13  9567 non-null   object
dtypes: int64(2), object(12)
memory usage: 1.2+ MB


In [4]:
df['타입'].value_counts()

AC완속               9656
DC차데모+AC3상+DC콤보     710
DC콤보                539
DC차데모+DC콤보          129
DC차데모+AC3상           29
AC3상                  9
DC차데모                 4
Name: 타입, dtype: int64

In [11]:
# AC3상이 급속인지 아닌지 찾기위해,AC3상에 해당하는 충전소를 검색해봅니다.
# 급속으로 분류되는 것을 확인 할수 있었습니다.
ac3 = df[df['타입']!='AC3상'].index
ac3df = df.drop(ac3)
ac3df.head()

,Unnamed: 0,운영기관,충전소,충전기 ID,타입,지역,시군구,주소,이용가능시간,제한,종류,비고,NaN,Unnamed: 13
1039,10817,한국전력,마곡엠밸리6단지,3,AC3상,서울특별시,강서구,서울특별시 강서구 마곡동 마곡서1로 100,NaN,시설 상황에 따라 이용이 제한될 수 있음,NaN,NaN,[1] 행정동 명칭,방화1동
3211,25795,차지비,서울시 금천구 롯대마트금천점,2,AC3상,서울특별시,금천구,서울특별시 금천구 시흥대로 291주차장(1155 금천롯데캐슬골드파크3차),24시간 이용가능,NaN,NaN,NaN,[1] 행정동 명칭,독산1동
4804,37544,지커넥트,대치SK뷰아파트,2,AC3상,서울특별시,강남구,"서울특별시 강남구 삼성로51길 25102동(3,5)B1기둥17 [4대] 102동(3...",24시간 이용가능,시설 상황에 따라 이용이 제한될 수 있음,NaN,NaN,NaN,NaN
5158,40047,지커넥트,신대림2차신동아파밀리에,1,AC3상,서울특별시,영등포구,서울특별시 영등포구 디지털로 420103동 지하 3층 1/4라인 출입구 부근 16번...,24시간 이용가능,거주자외 출입제한,NaN,NaN,NaN,NaN
5159,40048,지커넥트,신대림2차신동아파밀리에,2,AC3상,서울특별시,영등포구,서울특별시 영등포구 디지털로 420103동 지하 3층 1/4라인 출입구 부근 16번...,24시간 이용가능,거주자외 출입제한,NaN,NaN,NaN,NaN


In [33]:
df['이용가능시간'].value_counts(dropna=False)
# 117개 종류가 있는데 118개 다 보는 방법을 모르겠습니다.
# 기준이 너무 많으 것 같아서 데이터를 구별로 쪼개야 될것 같습니다.
# NaN 값은 찾아보니 아파트 단지 내에 존재하므로, 24시간으로 두겠습니다.
# 

24시간 이용가능                       5792
09:00~18:00                     1700
24시간 이용가능,입주민만 사용가능 거주자외출입제한     998
NaN                              804
24시간                             595
                                ... 
이용불가                               1
06:00~21:00                        1
13:00~17:00                        1
09:00~24:00                        1
23시간 이용(09:00~08:00)               1
Name: 이용가능시간, Length: 118, dtype: int64

In [ ]:
# 

In [136]:
def classify_gu(df,a):
  idx1 = df[df['시군구'] != a].index
  gu_df = df.drop(idx1)
  return gu_df 

In [57]:
def classify_fast(df):
  f_df = df[df['타입'] != 'AC완속']
  idxs = f_df.index
  s_df = df.drop(idxs)
  return f_df, s_df

In [106]:
# 이 함수에 기준은 구마다 다른 것 같아서 수정할 필요가 있습니다.
def classify_time(df):
  f1 = df[df['이용가능시간'] == '24시간 이용가능' ]
  f2 = df[df['이용가능시간'] == '주증/주말 : 24시간' ]
  f3 = df[df['이용가능시간'] == '24시간' ]
  f4 = df[df['이용가능시간'] == '~' ]
  f5 = df[df['이용가능시간'] == '00:00 ~ 23:59' ]
  f6 = df[df['이용가능시간'] == 'NaN' ]
  #이용 가능 시간 column에 시간 말고 다른 제한조건이 같이 있는 raw도 있습니다.
  c1 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자외출입제한' ]
  c2 = df[df['이용가능시간'] == '24시간 이용가능,입주민만 사용가능 거주자 외 출입제' ]

  full = pd.concat([f1,f2,f3,f4,f5,f6])
  temporary_c = pd.concat([c1, c2])
  fullfc = pd.concat([full, temporary_c])
  idx = fullfc.index
  nonfull = df.drop(idx)
  return full, nonfull, temporary_c

In [117]:
def classify_constraints(df,temporary_c):
  c1 = df[df['비고'] == '입주민만 사용가능 거주자외 출입제한' ]
  c2 = df[df['비고'] == '입주민만 사용가능 거주자 외 출입제한' ]
  c = pd.concat([temporary_c,c1,c2])
  nonc1 = df.drop(c1.index)
  nonc2 = nonc1.drop(c2.index)
  return c,nonc2

In [121]:
def classify_all(df,a):
  dfgu = classify_gu(df,a)
  # fast slow classify
  fast, slow = classify_fast(dfgu)
  # time classify
  fast_full , fast_nonfull , temporary_fast_cons = classify_time(fast) 
  slow_full , slow_nonfull , temporary_slow_cons = classify_time(slow)
  # constraints classify
  ffc, ffnc = classify_constraints(fast_full,temporary_fast_cons)
  nfc, nfnc = classify_constraints(fast_nonfull,temporary_fast_cons)
  fsc, fsnc = classify_constraints(slow_full, temporary_slow_cons)
  nsc, nsnc = classify_constraints(slow_nonfull, temporary_slow_cons)
  return ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc


In [58]:
# 
df = pd.read_excel('seoul_list.xlsx')
a = '동대문구 '

In [122]:
ffc,ffnc, nfc, nfnc, fsc, fsnc, nsc, nsnc = classify_all(df,a)

In [133]:
gu_df = pd.DataFrame()
gu_df['ffc'] =  ffc['Unnamed: 13'].value_counts(sort = True)
gu_df['ffnc'] =  ffnc['Unnamed: 13'].value_counts(sort = True)
gu_df['nfc'] =  nfc['Unnamed: 13'].value_counts(sort = True)
gu_df['nfnc'] =  nfnc['Unnamed: 13'].value_counts(sort = True)
gu_df['fsc'] =  fsc['Unnamed: 13'].value_counts(sort = True)
gu_df['fsnc'] =  fsnc['Unnamed: 13'].value_counts(sort = True)
gu_df['nsc'] =  nsc['Unnamed: 13'].value_counts(sort = True)
gu_df['nsnc'] =  nsnc['Unnamed: 13'].value_counts(sort = True)

In [138]:
gu_df.to_excel('dongdamoon_df.xlsx')
gu_df.head(10)

,ffc,ffnc,nfc,nfnc,fsc,fsnc,nsc,nsnc
청량리동,NaN,4,NaN,NaN,5.0,2.0,5.0,NaN
답십리2동,NaN,2,NaN,NaN,NaN,7.0,NaN,6.0
제기동,NaN,2,NaN,NaN,NaN,6.0,NaN,2.0
답십리1동,NaN,2,NaN,NaN,NaN,24.0,NaN,NaN
용신동,NaN,1,NaN,NaN,3.0,NaN,3.0,4.0
휘경2동,NaN,1,NaN,5.0,4.0,5.0,4.0,6.0
전농2동,NaN,1,NaN,2.0,8.0,25.0,8.0,1.0
전농1동,NaN,1,NaN,NaN,NaN,NaN,NaN,6.0
장안2동,NaN,1,NaN,NaN,NaN,12.0,NaN,8.0


,Unnamed: 0,운영기관,충전소,충전기 ID,타입,지역,시군구,주소,이용가능시간,제한,종류,비고,NaN,Unnamed: 13
5813,46648,에버온,동대문 용두롯데캐슬리치,1,AC완속,서울특별시,동대문구,"서울특별시 동대문구 한빛로 73 (용두동, 용두롯데캐슬리치)104동 B1 기둥47","24시간 이용가능,입주민만 사용가능 거주자외출입제한",NaN,NaN,입주민만 사용가능 거주자외 출입제한,[1] 행정동 명칭,용신동
5814,46649,에버온,동대문 용두롯데캐슬리치,2,AC완속,서울특별시,동대문구,"서울특별시 동대문구 한빛로 73 (용두동, 용두롯데캐슬리치)104동 B1 기둥47","24시간 이용가능,입주민만 사용가능 거주자외출입제한",NaN,NaN,입주민만 사용가능 거주자외 출입제한,[1] 행정동 명칭,용신동
5815,46650,에버온,동대문 용두롯데캐슬리치,3,AC완속,서울특별시,동대문구,"서울특별시 동대문구 한빛로 73 (용두동, 용두롯데캐슬리치)104동 B1 기둥47","24시간 이용가능,입주민만 사용가능 거주자외출입제한",NaN,NaN,입주민만 사용가능 거주자외 출입제한,[1] 행정동 명칭,용신동
6252,47912,에버온,서울동대문 장안위더스빌(B4),1,AC완속,서울특별시,동대문구,서울특별시 동대문구 한천로 42C동 B4 45기둥,"24시간 이용가능,입주민만 사용가능 거주자외출입제한",NaN,NaN,입주민만 사용가능 거주자외 출입제한,[1] 행정동 명칭,장안1동
6253,47913,에버온,서울동대문 장안위더스빌(B4),2,AC완속,서울특별시,동대문구,서울특별시 동대문구 한천로 42C동 B4 45기둥,"24시간 이용가능,입주민만 사용가능 거주자외출입제한",NaN,NaN,입주민만 사용가능 거주자외 출입제한,[1] 행정동 명칭,장안1동
